# Sistema de Reconhecimento Facial com MTCNN, FaceNet e SVM

Este notebook implementa um pipeline completo para reconhecimento facial, utilizando uma abordagem de três estágios:
1.  **Detecção de Faces:** Usando a rede pré-treinada **MTCNN** para encontrar a localização das faces em uma imagem.
2.  **Extração de Embeddings:** Usando a rede pré-treinada **FaceNet** para converter cada face em um vetor numérico de 128 dimensões que representa suas características únicas.
3.  **Classificação:** Treinando um classificador **Support Vector Machine (SVM)** para associar os vetores de embedding aos nomes das pessoas.

---

### Pré-requisito: Preparar a Base de Dados no Google Drive

Antes de executar o código, **recomendo fortemente** criar a seguinte estrutura de pastas e arquivos no seu Google Drive:

```
/content/drive/MyDrive/
└── reconhecimento_facial/             # Pasta principal do projeto
    ├── dataset/                     # Pasta com os dados para treinamento
    │   ├── nome_pessoa_1/           # Subpasta para a primeira pessoa
    │   │   ├── foto1.jpg
    │   │   ├── foto2.png
    │   │   └── ... (5 a 10 fotos da pessoa 1)
    │   └── nome_pessoa_2/           # Subpasta para a segunda pessoa
    │       ├── imagem_a.jpg
    │       └── ... (5 a 10 fotos da pessoa 2)
    └── imagem_teste.jpg             # Uma imagem de teste para a fase final
```

**Importante:** Os nomes das subpastas (`nome_pessoa_1`, etc.) serão usados como os rótulos para o reconhecimento.

### Fase 1: Instalação das Bibliotecas e Montagem do Drive

In [ ]:
# Instala as dependências necessárias
!pip install tensorflow keras opencv-python scikit-learn mtcnn keras-facenet

# Monta o Google Drive para acessar os arquivos
from google.colab import drive
drive.mount('/content/drive')

### Fase 2: Funções Principais e Inicialização dos Modelos

Nesta célula, importamos todas as bibliotecas, inicializamos os modelos pré-treinados (MTCNN para detecção e FaceNet para embeddings) e definimos a função principal que extrai a "assinatura" de uma face a partir de uma imagem.

In [ ]:
import numpy as np
from PIL import Image
import os
import pickle

# Importa os modelos e ferramentas
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Inicializa os modelos pré-treinados (isso pode levar um momento)
print("Carregando modelo de detecção MTCNN...")
detector = MTCNN()
print("Carregando modelo de embedding FaceNet...")
embedder = FaceNet()
print("Modelos carregados.")

def extrair_embedding(caminho_imagem):
    """
    Detecta a face em uma imagem e extrai seu embedding (vetor de características).
    
    Args:
        caminho_imagem (str): O caminho para o arquivo de imagem.
        
    Returns:
        np.ndarray: O vetor de embedding da face (128 dimensões), ou None se nenhuma face for encontrada.
    """
    try:
        img = Image.open(caminho_imagem)
    except Exception as e:
        print(f"Erro ao abrir a imagem {caminho_imagem}: {e}")
        return None

    img_rgb = img.convert('RGB')
    pixels = np.asarray(img_rgb)
    
    resultados = detector.detect_faces(pixels)
    
    if resultados:
        x1, y1, width, height = resultados[0]['box']
        # Lida com coordenadas negativas, que podem ocorrer
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        
        face_pixels = pixels[y1:y2, x1:x2]
        face_img = Image.fromarray(face_pixels)
        face_img = face_img.resize((160, 160))
        face_array = np.asarray(face_img)
        
        face_array = face_array.astype('float32')
        mean, std = face_array.mean(), face_array.std()
        face_array = (face_array - mean) / std
        face_array = np.expand_dims(face_array, axis=0)
        
        embedding = embedder.embeddings(face_array)
        return embedding[0]
        
    return None

### Fase 3: Carregar Dataset e Treinar o Classificador SVM

Agora, vamos processar todas as imagens do nosso dataset. Para cada imagem, extraímos o embedding da face e o associamos ao nome da pessoa (o nome da pasta). No final, treinamos um modelo SVM com esses dados e o salvamos.

In [ ]:
# ATENÇÃO: Verifique se este caminho está correto!
caminho_dataset = '/content/drive/MyDrive/reconhecimento_facial/dataset/'

embeddings = []
labels = []

print("Iniciando processamento do dataset...")
for nome_pessoa in os.listdir(caminho_dataset):
    caminho_pessoa = os.path.join(caminho_dataset, nome_pessoa)
    
    if os.path.isdir(caminho_pessoa):
        for nome_imagem in os.listdir(caminho_pessoa):
            caminho_imagem = os.path.join(caminho_pessoa, nome_imagem)
            
            embedding = extrair_embedding(caminho_imagem)
            
            if embedding is not None:
                embeddings.append(embedding)
                labels.append(nome_pessoa)
                print(f"Processado: {caminho_imagem}")

if embeddings:
    embeddings = np.asarray(embeddings)
    labels = np.asarray(labels)

    # Normaliza os vetores de embedding
    in_encoder = Normalizer(norm='l2')
    embeddings = in_encoder.transform(embeddings)

    # Codifica os rótulos para números
    out_encoder = LabelEncoder()
    out_encoder.fit(labels)
    labels_encoded = out_encoder.transform(labels)

    # Divide os dados para treino e teste (80% treino, 20% teste)
    X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_encoded, test_size=0.2, random_state=42)

    # Treina o modelo SVM
    modelo_svm = SVC(kernel='linear', probability=True)
    modelo_svm.fit(X_train, y_train)

    # Avalia o modelo
    y_pred = modelo_svm.predict(X_test)
    print(f"\nAcurácia do modelo nos dados de teste: {accuracy_score(y_test, y_pred) * 100:.2f}%")

    # Salva o modelo treinado e o codificador de rótulos
    caminho_salvar = '/content/drive/MyDrive/reconhecimento_facial/'
    with open(os.path.join(caminho_salvar, 'svm_classifier.pkl'), 'wb') as f:
        pickle.dump(modelo_svm, f)
    with open(os.path.join(caminho_salvar, 'label_encoder.pkl'), 'wb') as f:
        pickle.dump(out_encoder, f)

    print(f"\nModelo treinado e salvo com sucesso em: {caminho_salvar}")
else:
    print("\nNenhuma face foi processada. Verifique sua estrutura de pastas e as imagens.")

### Fase 4: Teste Final em uma Nova Imagem

Esta célula final carrega os modelos que acabamos de salvar e executa o pipeline completo em uma imagem de teste. Ele irá detectar todas as faces, reconhecê-las e desenhar o resultado.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# --- Carregar modelos e dados salvos ---
caminho_base = '/content/drive/MyDrive/reconhecimento_facial/'

try:
    with open(caminho_base + 'svm_classifier.pkl', 'rb') as f:
        modelo_svm = pickle.load(f)
    with open(caminho_base + 'label_encoder.pkl', 'rb') as f:
        out_encoder = pickle.load(f)
except FileNotFoundError:
    print("Erro: Arquivos de modelo não encontrados. Certifique-se de que a Fase 3 foi executada com sucesso.")

in_encoder = Normalizer(norm='l2')

# --- Carregar imagem de teste ---
# ATENÇÃO: Altere o nome do arquivo para a sua imagem de teste!
caminho_imagem_teste = '/content/drive/MyDrive/reconhecimento_facial/imagem_teste.jpg'

try:
    img_teste = cv2.imread(caminho_imagem_teste)
    img_rgb = cv2.cvtColor(img_teste, cv2.COLOR_BGR2RGB)
    pixels = np.asarray(img_rgb)

    # Detectar todas as faces na imagem
    faces = detector.detect_faces(pixels)

    if faces:
        print(f"{len(faces)} face(s) detectada(s).")
        for face_info in faces:
            x1, y1, width, height = face_info['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            
            face_pixels = pixels[y1:y2, x1:x2]
            face_img = Image.fromarray(face_pixels).resize((160, 160))
            face_array = np.asarray(face_img).astype('float32')
            
            mean, std = face_array.mean(), face_array.std()
            face_array = (face_array - mean) / std
            face_array = np.expand_dims(face_array, axis=0)
            
            embedding = embedder.embeddings(face_array)
            embedding_norm = in_encoder.transform(embedding)
            
            # Fazer a predição
            predicao_numerica = modelo_svm.predict(embedding_norm)
            probabilidade = modelo_svm.predict_proba(embedding_norm)
            confianca = np.max(probabilidade)
            
            nome_predito = out_encoder.inverse_transform(predicao_numerica)[0]
            
            # Se a confiança for baixa, classificar como "Desconhecido"
            if confianca < 0.85:
                nome_predito = "Desconhecido"
                cor_retangulo = (0, 0, 255) # Vermelho para desconhecido
            else:
                cor_retangulo = (0, 255, 0) # Verde para conhecido
            
            # Desenhar retângulo e texto na imagem original
            cv2.rectangle(img_teste, (x1, y1), (x2, y2), cor_retangulo, 2)
            texto = f'{nome_predito} ({confianca*100:.2f}%)'
            cv2.putText(img_teste, texto, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, cor_retangulo, 2)

        # Exibir o resultado
        print("\nExibindo resultado...")
        cv2_imshow(img_teste)
    else:
        print("Nenhuma face foi detectada na imagem de teste.")
except FileNotFoundError:
    print(f"Erro: Imagem de teste não encontrada em '{caminho_imagem_teste}'. Verifique o caminho e o nome do arquivo.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")